# THIS IS THE ORIGINAL DANIEL NOTEBOOK

# Purpose of notebook: to make txt files with sources and text body into csv files and merge them 

In [14]:
# import/load packages

## install
%pip install pandas
%pip install chardet
%pip install cchardet


## import
import os
import pandas as pd
import chardet
import cchardet
import re
from bs4 import BeautifulSoup

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## From TXT to CSV (+ global preprocessing) - sources-17-05.txt + text_17-05-US.txt


### sources-17-05.txt

In [15]:
# define file path
file_path = "/work/Bachelor/sample_data/dropbox/sources-17-05.txt"

# read csv file into dataframe
df_sources_17_05 = pd.read_csv(file_path, delimiter='\t', encoding='ISO-8859-1')

# define column names
column_names = ["textID", "words", "date", "country", "source", "url", "title"]

# assign new column names to the dataframe
df_sources_17_05.columns = column_names
df_sources_17_05

,textID,words,date,country,source,url,title
0,18207168,2362,17-05-01,US,Huffington Post,http://www.huffingtonpost.com/entry/a-letter-t...,A letter to parents of expat children
1,18207169,343,17-05-01,US,GoLocalProv,http://www.golocalprov.com/business/chafee-ste...,Chafee Steps Up Criticism of Raimondo's Econom...
2,18207170,257,17-05-01,US,Engadget,https://www.engadget.com/2017/05/01/lab-grown-...,Lab-grown stem cells may carry an increased ri...
3,18207172,180,17-05-01,US,Northwest Herald,http://www.nwherald.com/2017/04/28/friends-of-...,Friends of McHenry County College Foundation t...
4,18207174,630,17-05-01,US,GoLocalProv,http://www.golocalprov.com/politics/guest-mind...,Guest MINDSETTERT James Monteiro: Vote No in R...
...,...,...,...,...,...,...,...
297859,18740662,304,17-05-31,JM,Jamaica Star Online,http://jamaica-star.com/article/news/20170531/...,Christopher Townsend is back on Munga's murder...
297860,18742209,315,17-05-31,JM,Jamaica Gleaner,http://jamaica-gleaner.com/article/news/201705...,OUR proposes fixes to JPS after August 2016 bl...
297861,18742210,127,17-05-31,JM,Jamaica Star Online,http://jamaica-star.com/article/entertainment/...,Journalist curses Damian Marley in Kenya
297862,18742211,288,17-05-31,JM,Jamaica Star Online,http://jamaica-star.com/article/entertainment/...,DON'T PRAY FOR ME ... Pepita slams unworthy Ch...


In [16]:
# check how many articles there are from each country
country_counts = df_sources_17_05['country'].value_counts()
print(country_counts)

country
IN    31756
CA    30205
US    26663
GB    26223
IE    25204
NG    21151
AU    20726
NZ    18999
ZA    18324
MY    17759
SG    16959
PH    14018
PK    10431
KE     7038
GH     6066
HK     1853
LK     1762
JM     1352
BD     1279
TZ       96
Name: count, dtype: int64


In [17]:
# filter to only include sources from the US
df_sources_17_05_US = df_sources_17_05[df_sources_17_05['country'] == 'US']

# check length to compare with the dropbox_text file
len(df_sources_17_05_US)

26663

### text_17-05-US.txt

In [18]:
# define file path
file_path = "/work/Bachelor/sample_data/dropbox/text_17-05-US.txt"

# open the file and read its contents into a string
with open(file_path, 'r', encoding='utf-8') as file:
    raw_text = file.read()  # Read all text data from the file

sample = raw_text
sample = re.sub(" ([.,?!':])", r"\1", sample) # punctuation characters are all needlessly preceded by a space, remove that space
sample = re.sub("@ @ @ @ @ @ @ @ @ @", "CENSORED", sample) # replace the keyword chosen to mark words or entities that have been censored

# Step 1: Split the text into articles based on '@@' markers
article_ids = re.findall(r"@@(\d+)", sample) # extract list of all article IDs by matching pattern of any sequence of digits following exactly two @
articles = re.split(r'"?@@\d+ ', sample)[1:] # split articles on article IDs, i.e. a sequence of digits preceded by exactly two @s and sometimes a quotation mark preceding that
articles = [art[art.find("<p> ") + 4:].strip().replace(" <p> ", "\n") for art in articles] # for each article, skip to the first paragraph tag to exclude heading/title text, then replace subsequent markers for new paragraphs with line breaks. OBS maybe 'words' column contains incorrect n words because of this. 

print(len(article_ids), len(articles)) # check that we found equally many IDs and article bodies

df_articles_17_05_US = pd.DataFrame(data = dict(textID = article_ids, body = articles)) # create dataframe with extracted IDs and corresponding article contents
df_articles_17_05_US["textID"] = df_articles_17_05_US["textID"].astype(int) # cast textIDs to int to enable merging with `sources` dataframe
df_articles_17_05_US.sample(10) # sample ten random rows from the dataframe to examine whether things look as expected


27087 27087


,textID,body
5564,18309968,"NEW YORK -- May the 4th be with you, Laura Der..."
22854,18659200,Please Wait <h> Thank You for Subscribing!\nYo...
1952,18243883,"The hospital block, also known as Cellblock 3,..."
12299,18438729,TORONTO -- They got dressed mostly in silence ...
12129,18435730,"Please log in, sign up for a new account and p..."
24019,18678909,Our voicemail system is currently down and is ...
16766,18534376,"THE RUNDOWN WACO, Texas -- Baylor baseball dro..."
22146,18644306,"WEST BRANDYWINE, Pa. -- Two counselors at a Pe..."
153,18209157,Bruce Kerfoot will never forget standing on hi...
15403,18506570,"BEIRUT, LEBANON ( 2:30 P.M. ) -- The Islamic S..."


In [19]:
joined_17_05 = df_sources_17_05_US.merge(df_articles_17_05_US, on = "textID", how = "left") # join sources dataframe with their corresponding article content using the textIDs, keeping only the articles where source information was available
joined_17_05

,textID,words,date,country,source,url,title,body
0,18207168,2362,17-05-01,US,Huffington Post,http://www.huffingtonpost.com/entry/a-letter-t...,A letter to parents of expat children,"Parenting is fraught with anxiety, joys, chall..."
1,18207169,343,17-05-01,US,GoLocalProv,http://www.golocalprov.com/business/chafee-ste...,Chafee Steps Up Criticism of Raimondo's Econom...,In a letter sent to members of the Board of th...
2,18207170,257,17-05-01,US,Engadget,https://www.engadget.com/2017/05/01/lab-grown-...,Lab-grown stem cells may carry an increased ri...,If you've followed the latest medical research...
3,18207172,180,17-05-01,US,Northwest Herald,http://www.nwherald.com/2017/04/28/friends-of-...,Friends of McHenry County College Foundation t...,The Friends of McHenry County College Foundati...
4,18207174,630,17-05-01,US,GoLocalProv,http://www.golocalprov.com/politics/guest-mind...,Guest MINDSETTERT James Monteiro: Vote No in R...,I write to go on the record and say that I've ...
...,...,...,...,...,...,...,...,...
26658,18745486,1212,17-05-31,US,OCRegister,http://www.ocregister.com/2017/05/31/rememberi...,Remembering Marilyn Monroe on anniversary of h...,1954: American film star Marilyn Monroe ( 1926...
26659,18745488,424,17-05-31,US,fox4kc.com,http://fox4kc.com/2017/05/31/on-the-two-year-a...,"On the two-year anniversary of his murder, Kan...","RAYTOWN, Mo. -- A 3-year-old's siblings and mo..."
26660,18745489,405,17-05-31,US,FOX 5 Atlanta,http://www.fox5atlanta.com/news/258189601-story,Artists file lawsuit against city of Atlanta o...,ATLANTA - A group of artists have drawn a lega...
26661,18745490,458,17-05-31,US,Q13 FOX,http://q13fox.com/2017/05/31/man-accused-of-ba...,"Man accused of backing truck over 2 men, killi...","MONTESANO, Wash. -- James Walker, 31, of Hoqui..."


### It works!!
### OBS there are fewer rows in sources (26,663) than in text (27,087), which can be caused by a) sources isn't split correctly, b) sources are simply missing from the dataset hence fewer rows. Consider investigating this.